In [1]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import time
from pyluos import Device
from IPython.display import clear_output
import numpy as np

device = Device('/dev/cu.usbserial-DN2JWZ3D')
print(device.services)
    
def plot_test_cmplt(baudrate_vector, results):
    fig1 = plt.figure(1)
    ax = plt.subplot(1, 1, 1)
    ax.set_xlabel('Packet size (bytes)')
    ax.set_ylabel('Data rate (bits/s)')
    for i,baud in enumerate(baudrate_vector):
        ax.plot(results[i]["datasize_vector"], results[i]["data_rate"], label=str(baud) + "baub")
    ax.legend(loc='upper right')
    fig2 = plt.figure(2)
    ax1 = plt.subplot(1, 1, 1)
    ax1.set_xlabel('Packet size (bytes)')
    ax1.set_ylabel('Failed rate (%)')
    for i,baud in enumerate(baudrate_vector):
        ax1.plot(results[i]["datasize_vector"], results[i]["fail_rate"], label=str(baud) + "baub")
    ax1.legend(loc='upper right')
    plt.show()

def data_rate_test(baud):
    datarate = []
    failrate = []
    datasize = []
    device.baudrate = baud
    maxsize = 256
    data_size = range(1, maxsize, 1)
    total = 0
    for size in data_size: 
        clear_output(wait=True)
        data = [12]
        data *= size
        results = device.benchmark(data=data, repetition=200, target_id=2)
        total = total + (size*200*(100.0-results[1])/100/1000)
        if total < 1000 :
            print ("baudrate : " + str(baud) + "\tpayload size : " + str(size*200/1000) + " kB\ttotal transfered : " + str(total) +  " kB\n[" + "=" * int(size*100/maxsize) + " " * int(100-(size*100/maxsize)) + "]")
        else :
            print ("baudrate : " + str(baud) + "\tpayload size : " + str(size*200/1000) + " kB\ttotal transfered : " + str(total/1000) +  " mB\n[" + "=" * int(size*100/maxsize) + " " * int(100-(size*100/maxsize)) + "]")
        datarate.append(results[0])
        failrate.append(results[1])
        datasize.append(size)
    return {"datasize_vector" : datasize, "data_rate" : datarate, "fail_rate" : failrate}

def benchmark():
    baudrates = [250000, 500000, 1000000, 2000000]
    results = []
    device.pause()
    for baud in baudrates:
        results.append(data_rate_test(baud))
    plot_test_cmplt(baudrates, results)
    device.play()


Connected to "/dev/cu.usbserial-DN2JWZ3D".
Sending detection signal.
Waiting for routing table...
Device setup.
-------------------------------------------------
Type                Alias               ID   
-------------------------------------------------
Gate                gate                1    
Color               rgb_led_mod         2    



In [ ]:
benchmark()

baudrate : 2000000	payload size : 45.0 kB	total transfered : 0.0 kB
[=======================================================================================            ]
